In [ ]:
# Step 0. Load libraries and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Step 1. Load the dataset
# Source: http://lib.stat.cmu.edu/datasets/boston
# The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic
# prices and the demand for clean air', J. Environ. Economics & Management,
# vol.5, 81-102, 1978.   Used in Belsley, Kuh & Welsch, 'Regression diagnostics
# ...', Wiley, 1980.   N.B. Various transformations are used in the table on
# pages 244-261 of the latter.
# Attribute Information (in order):
#        - CRIM     per capita crime rate by town
#        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
#        - INDUS    proportion of non-retail business acres per town
#        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
#        - NOX      nitric oxides concentration (parts per 10 million)
#        - RM       average number of rooms per dwelling
#        - AGE      proportion of owner-occupied units built prior to 1940
#        - DIS      weighted distances to five Boston employment centres
#        - RAD      index of accessibility to radial highways
#        - TAX      full-value property-tax rate per $10,000
#        - PTRATIO  pupil-teacher ratio by town
#        - LSTAT    % lower status of the population
#        - MEDV     Median value of owner-occupied homes in $1000's
df_raw = pd.read_csv('data/raw/boston.csv')

In [ ]:
# 1.1 Get basic info


In [ ]:
# 1.2 Take a sample


In [ ]:
# 1.3 Transform if needed



In [ ]:
# Step 2. Make a basic EDA

In [ ]:
# 2.1 Split the dataset so to avoid bias


In [ ]:
# 2.2 Join the train sets to ease insights


In [ ]:
# 2.3 Get basic info


In [ ]:
# 2.4 Take a sample


In [ ]:
# 2.5 Describe the numerical and date variables


In [ ]:
# 2.6 Describe the categorical variables

In [ ]:
# 2.7 Perform univariate analysis - histograms


In [ ]:
# 2.8 Perform univariate analysis - boxplots


In [ ]:
# 2.9 Perform bivariate analysis - scatter matrix


In [ ]:
# 2.10 Perform correlation analysis - Pearson or Point Biserial


In [ ]:
# 2.11 Perform multivariate analysis - VIF

In [ ]:
# Step 3. Create and evaluate model - Define your approach and goals

In [ ]:
# 3.1 Perform model fit - A


In [ ]:
# 3.2 Perform model fit - B


In [ ]:
# 3.3 Calculate metrics on model A

In [ ]:
# 3.4 Calculate metrics on model B

In [ ]:
# Step 4. Write up your conclusions